# Distinguishing Traffic Signs

## Preparing image data

In [1]:
URL <- "https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip"
download.file(url = URL, destfile = "GTSRB_Final_Training_Images.zip", mode='wb')

Warning message in download.file(url = URL, destfile = "GTSRB_Final_Training_Images.zip", :
“downloaded length 97599488 != reported length 276294756”
Warning message in download.file(url = URL, destfile = "GTSRB_Final_Training_Images.zip", :
“URL 'https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip': Timeout of 60 seconds was reached”


ERROR: Error in download.file(url = URL, destfile = "GTSRB_Final_Training_Images.zip", : download from 'https://sid.erda.dk/public/archives/daaeac0d7ce1152aea9b61d9f1e19370/GTSRB_Final_Training_Images.zip' failed


In [2]:
library(utils)
zipF<- "GTSRB_Final_Training_Images.zip"
outDir<-"."
unzip(zipF, exdir=outDir)

Warning message in unzip(zipF, exdir = outDir):
“error 1 in extracting from zip file”


In [1]:
images <- list.files(pattern = "*.ppm$", recursive = TRUE)

In [2]:
set.seed(0)
sample_size <- floor(0.67 * length(images))
train_idx <- sample(seq_len(length(images)), size = sample_size)

In [3]:
train_images <- images[train_idx]
test_images <- images[-train_idx]

In [6]:
dir.create("traffic_signs_train")
dir.create("traffic_signs_test")

In [6]:
cl_extract <- function(x) {
    return(as.integer(strsplit(x, "/")[[1]][4]))
}

cl_train <- as.numeric(sapply(train_images, cl_extract))
cl_test <- as.numeric(sapply(test_images, cl_extract))

In [7]:
counts <- table(cl_train)
weights <- as.list((sum(counts) - counts) / counts)

In [9]:
for (u in unique(c(cl_train, cl_test))) {
    dir.create(paste("traffic_signs_train/", u))
    dir.create(paste("traffic_signs_test/", u))
}

In [10]:
for (f in 1:length(cl_train)) {
    file.copy(train_images[f], paste("traffic_signs_train/", cl_train[f]))
}

In [11]:
for (f in 1:length(cl_test)) {
    file.copy(test_images[f], paste("traffic_signs_test/", cl_test[f]))
}

In [12]:
unlink("./GTSRB", recursive=TRUE)

## Running a classification task

In [8]:
if (!("keras" %in% rownames(installed.packages()))) {
    install.packages("devtools")
    devtools::install_github("rstudio/keras")
    reticulate::py_config()
}

In [9]:
library(keras)

batch_size=256

training_image_gen <- image_data_generator(
  rotation_range = 5,
  width_shift_range = 0.1,
  height_shift_range = 0.1,
  horizontal_flip = FALSE,
  rescale = 1./255.
)

validation_image_gen <- image_data_generator(
  rescale = 1./255.
)

IMG_SIZE <- 32

training_image_flow <- flow_images_from_directory(
  directory = "./traffic_signs_train/", 
  generator = training_image_gen, 
  class_mode = "categorical",
  batch_size = batch_size,
  target_size = c(IMG_SIZE, IMG_SIZE), 
)

validation_image_flow <- flow_images_from_directory(
  directory = "./traffic_signs_test/", 
  generator = validation_image_gen, 
  class_mode = "categorical",
  batch_size = batch_size,
  target_size = c(IMG_SIZE, IMG_SIZE), 
  shuffle = FALSE
)

In [10]:
NUM_CLASSES <- length(unique(c(cl_train, cl_test)))

model <- keras_model_sequential() %>% 
  layer_conv_2d(filters=32, kernel_size=c(5, 5), padding='same',
                     input_shape=c(IMG_SIZE, IMG_SIZE, 3),
                     activation='relu') %>%
  layer_conv_2d(filters=64, kernel_size=c(5, 5), activation='relu') %>%
  layer_flatten() %>%
  layer_dense(768, activation="relu") %>% 
  layer_dropout(rate=0.4) %>% 
  layer_dense(units=NUM_CLASSES, activation = "sigmoid")

model

Model
________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
conv2d (Conv2D)                     (None, 32, 32, 32)              2432        
________________________________________________________________________________
conv2d_1 (Conv2D)                   (None, 28, 28, 64)              51264       
________________________________________________________________________________
flatten (Flatten)                   (None, 50176)                   0           
________________________________________________________________________________
dense (Dense)                       (None, 768)                     38535936    
________________________________________________________________________________
dropout (Dropout)                   (None, 768)                     0           
________________________________________________________________________________
dense_1 (Dense)       

In [ ]:
model %>% 
  compile(loss="categorical_crossentropy", optimizer="adam", metrics="accuracy")

history <- model %>% fit_generator(
              generator = training_image_flow, 
              epochs = 10, 
              steps_per_epoch = training_image_flow$n/training_image_flow$batch_size,
              validation_data = validation_image_flow,
              validation_steps = validation_image_flow$n/validation_image_flow$batch_size,
              class_weight = weights
)

In [ ]:
history

In [ ]:
# Plotting training and test dynamics
plot(history, method='auto', metrics=c('loss', 'categorical_accuracy'))